In [ ]:
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
import tweepy
import json
import pandas as pd
import csv
import re
from textblob import TextBlob
import string
import os
import time
# Twitter credentials
# Obtain them from your twitter developer account
consumer_key = 'DNvrqniGAvAaSVqfwI1nd9Iy1'
consumer_secret = '8xma1rxgBWBzsOwxl02XuB1vBrjgPXGJgCeCLXfkQIuEeJpqbE'
access_key = '1499145505481404421-eUvIVTJf8CfmzMu248iK5wPRxajD9a'
access_secret = 'ESMAC8i3a9JWsTrMDqX9DDnjR8mwBoj9qWvlee3JeNDYp'


In [ ]:
#Twitter authentication

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)   
auth.set_access_token(access_key, access_secret) 
  
# Creating an API object 
api = tweepy.API(auth)
def  authenticate_user():

  try:
    api.verify_credentials()
    print("Authentication with twitter successfull")
  except:
    print("Authentication failed")

In [ ]:
def writeToCSV(db_tweets, type_of_tweet):  
    from datetime import datetime
    to_csv_timestamp = datetime.today().strftime('%Y%m%d_%H%M%S')

    path = os.getcwd()
    filename = 'Tweets'+'_'+type_of_tweet+'.csv'

    db_tweets.to_csv(filename, index = False)

In [ ]:
def scrapTweetsByTags(search_words, date_since, numTweets, numRuns):

    db_tweets = pd.DataFrame(columns = ['username',  'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags'])
    
    for i in range(0, numRuns):
        start_run = time.time()
        tweets = tweepy.Cursor(api.search, q=search_words, lang="en", until="2022-06-02", tweet_mode='extended', wait_on_rate_limit=True).items(numTweets)
        tweet_list = [tweet for tweet in tweets]
        noTweets = 0

        for tweet in tweet_list:
            username = tweet.user.screen_name
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']

            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text

            ith_tweet = [username, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

            db_tweets.loc[len(db_tweets)] = ith_tweet
            noTweets += 1
        
        end_run = time.time()
        duration_run = round(end_run-start_run, 2)
        
        print('no. of tweets scraped for run {} is {}'.format(i, noTweets))
        print('time take for {} run to complete is {}'.format(i, duration_run))
        
        #time.sleep(2) #15 minute sleep time
    return db_tweets
    print('Scraping has completed!')

In [ ]:
def scrapTweetsByUser(user_name, search_words, date_since, numTweets, numRuns):

    db_tweets_user = pd.DataFrame(columns = ['username', 'location', 'following',
                                        'followers', 'totaltweets', 'usercreatedts', 'tweetcreatedts',
                                        'retweetcount', 'text', 'hashtags'])
    
    for user in user_name:
        start_run = time.time()
        tweets = tweepy.Cursor(api.user_timeline,screen_name = user ,lang="en", since="2020-01-01",until="2020-01-02",  tweet_mode='extended', wait_on_rate_limit=True).items(numTweets)
        tweet_list = [tweet for tweet in tweets]
        noTweets = 0

        for tweet in tweet_list:
            username = tweet.user.screen_name
            location = tweet.user.location
            following = tweet.user.friends_count
            followers = tweet.user.followers_count
            totaltweets = tweet.user.statuses_count
            usercreatedts = tweet.user.created_at
            tweetcreatedts = tweet.created_at
            retweetcount = tweet.retweet_count
            hashtags = tweet.entities['hashtags']

            try:
                text = tweet.retweeted_status.full_text
            except AttributeError:  # Not a Retweet
                text = tweet.full_text

            ith_tweet = [username, location, following, followers, totaltweets,
                         usercreatedts, tweetcreatedts, retweetcount, text, hashtags]

            db_tweets_user.loc[len(db_tweets_user)] = ith_tweet
            noTweets += 1
        
        end_run = time.time()
        duration_run = round(end_run-start_run, 2)
        
        print('no. of tweets scraped for USER {} is {}'.format(user, noTweets))
        print('time take for {} run to complete is {}'.format(user, duration_run))
        
        #time.sleep(5) #15 minute sleep time
    return db_tweets_user
    print('Scraping has completed!')

In [ ]:
search_words = "#AMZN OR $AMZN OR  #amzn OR #techstocks OR #stockmarket OR #NYSE OR #investing OR #Stocks"
#search_words = "#NFLX"
#user_name = ["CNBC","Benzinga","Stocktwits","SJosephBurns","BreakoutStocks","bespokeinvest","WSJMarkets","Stephanie_Link","nytimesbusiness","IBDinvestors","WSJDealJournal","MarketWatch"]
user_name = ["CNBC"]
date_since = "2000-01-01"
filter_retweets =" -filter:retweets"
numTweets = 20000
numRuns =1


authenticate_user()
#tweets =  scrapTweetsByTags(search_words, date_since, numTweets, numRuns)
tweets =  scrapTweetsByUser(user_name,search_words, date_since, numTweets, numRuns)
writeToCSV(tweets,'Tags')

Authentication with twitter successfull
no. of tweets scraped for USER CNBC is 280
time take for CNBC run to complete is 5.59
